In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor(),
)

100.0%


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [3]:
batch_size=64

train_dataloader = DataLoader(training_data,
                              batch_size=batch_size)
test_dataloader = DataLoader(test_data,
                             batch_size=batch_size)

for X, y in test_dataloader:
    print(X.shape)
    print(y.shape, y.dtype)
    break

torch.Size([64, 1, 28, 28])
torch.Size([64]) torch.int64


In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using device', device)

Using device cpu


In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),
                            lr=1e-3)

In [12]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [13]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= num_batches
        correct /= size
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [14]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n---------------------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print('Done!')

Epoch 1
---------------------------------------------
loss: 1.149385 [    0/60000]
loss: 1.157321 [ 6400/60000]
loss: 0.975167 [12800/60000]
loss: 1.107067 [19200/60000]
loss: 0.978475 [25600/60000]
loss: 1.020018 [32000/60000]
loss: 1.058015 [38400/60000]
loss: 0.989444 [44800/60000]
loss: 1.030879 [51200/60000]
loss: 0.970079 [57600/60000]
Test Error: 
 Accuracy: 65.6%, Avg loss: 0.973610 

Epoch 2
---------------------------------------------
loss: 1.031317 [    0/60000]
loss: 1.059581 [ 6400/60000]
loss: 0.861683 [12800/60000]
loss: 1.015270 [19200/60000]
loss: 0.888397 [25600/60000]
loss: 0.927099 [32000/60000]
loss: 0.980772 [38400/60000]
loss: 0.917940 [44800/60000]
loss: 0.954437 [51200/60000]
loss: 0.903041 [57600/60000]
Test Error: 
 Accuracy: 67.0%, Avg loss: 0.902247 

Epoch 3
---------------------------------------------
loss: 0.945772 [    0/60000]
loss: 0.992000 [ 6400/60000]
loss: 0.781347 [12800/60000]
loss: 0.950800 [19200/60000]
loss: 0.828594 [25600/60000]
loss: 0.8

In [15]:
torch.save(model.state_dict(), 'model.pth')
print('Saved PyTorch Model State to model.pth')

Saved PyTorch Model State to model.pth


In [16]:
model = NeuralNetwork()
model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

In [17]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
